## Create note table

This file take all the csv files that contain a note attribut, exctract all the notes, remove all the duplicates, then we create a dictionnary that associated each note to an id (id is the vale and the note is the key). We then save everything in a csv file


In [57]:
import pandas as pd
import os

path = '../data/original/'
data_dict = {}

for file in os.listdir(path):
    name = file.split('.')[0]
    data_dict[name] = pd.read_csv(path + file,error_bad_lines=False)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 6010: expected 16 fields, saw 29\nSkipping line 14907: expected 16 fields, saw 29\n'
b'Skipping line 142947: expected 16 fields, saw 27\n'
b'Skipping line 379178: expected 16 fields, saw 27\n'
b'Skipping line 411105: expected 16 fields, saw 18\n'
b'Skipping line 532885: expected 16 fields, saw 27\n'
b'Skipping line 625092: expected 16 fields, saw 27\n'
b'Skipping line 743624: expected 16 fields, saw 27\n'
b'Skipping line 786680: expected 16 fields, saw 27\n'
b'Skipping line 892034: expected

In [60]:

files = ['story.csv','issue.csv','series.csv','indicia_publisher.csv','publisher.csv','brand_group.csv']
notes = pd.Series()
for key,df in data_dict.items():
    try: 
        note = df['notes'].dropna()
        notes = pd.concat([notes,note],ignore_index =True)
    except:
        continue  
    
notes.drop_duplicates(keep='first',inplace = True)
notes

0         Used for the MLJ superheroes that DC licensed ...
1         An imprint of HarperCollins Publishers; URL li...
2         Lancé en 2008 le label Fusion Comics était com...
3         a highly-stylized "A" with the crossbar formed...
4                     Letter B is larger than rest of text.
5                                Used at least during 1970.
6             also includes silhouette drawing of a rooster
7         This brand was used on one-shots produced by T...
8         Picture of the Capitol building with the words...
9         Words "A Century Publication" in a circle arou...
10        The words "A Comics Interview Publication" ins...
11        Emblem has "Dell" in the center of a square ma...
12        Same emblem as used on the Dell comics the com...
13        Appeared on Dime Comics but definitely as a ma...
14        Rather than create a new brand Phillips used t...
15        Brand used only on ACG American Comics Group's...
16        The first part of the brand is

In [49]:
notes = notes.reset_index(drop=True)
notes.to_dict()

notes_dict = {v: k for k, v in notes.to_dict().items()}
notes_dict

{'Lettering credit from Dick Ayers via Mike Quilligan via the GCD Errors List.Inks credit from Jim Vadaboncoeur Jr via Atlas Tales (http://www.atlastales.com/issue/3420) 2015-06-14; the previous indexer and Tom Lammers credit the inks to Ayers.': 0,
 'Lettering credit from Dick Ayers via Mike Quilligan via the GCD Errors List.': 1,
 'Single panel cartoons': 2,
 'Victorian style comic (no word balloons; brief text beneath each picture)': 3,
 'single panel cartoons': 4,
 'one full page single panel cartoon': 5,
 'Victorian style comic -- short text beneath each picture no word balloons': 6,
 'Cover is all text.': 7,
 'Victorian style comic -- rhyming narration beneath each picture with only one word balloon used for emphasis.': 8,
 'Victorian style comic: text narration appears beneath each picture no word balloons.': 9,
 'No real "cover" -- the two stories below essentially make up the front & back covers with the interior blank or text.': 10,
 'Victorian style comic -- brief text narra